In [1]:
import json
import numpy as np
import pandas as pd
import string

In [2]:
SPAM = True
NOT_SPAM = False

In [3]:
# Пробуем открыть файл со справочником слов, встречаемых в спам-текстах
try:
    spam_words = json.load('spam_words.json')
# Если такого файла еще нет, то создаем пустой справочник
except:
    spam_words = dict()

# Пробуем открыть файл со справочником слов, встречаемых в хороших текстах
try:
    good_words = json.load('good_words.json')
# Если такого файла еще нет, то создаем пустой справочник
except:
    good_words = dict()

# Пробуем открыть файл со счетчиками слов
try:
    words_count = json.load('words_count.json')
# Если такого файла еще нет, то создаем справочник с нулевыми значениями
except:
    words_count = {
        'spam': 0,
        'good': 0
    }

# Пробуем открыть файл с вероятностями
try:
    probs = json.load('probs.json')
# Если такого файла еще нет, то создаем справочник с нулевыми значениями
except:
    probs = {
        'spam_texts_count': 0,
        'good_texts_count': 0,
        'pA': 0,
        'pNotA': 0,
    }

In [4]:
# train_data = [  spam_or_not_spam.csv
#     ['Купите новое чистящее средство', SPAM],   
#     ['Купи мою новую книгу', SPAM],  
#     ['Подари себе новый телефон', SPAM],
#     ['Добро пожаловать и купите новый телевизор', SPAM],
#     ['Привет давно не виделись', NOT_SPAM], 
#     ['Довезем до аэропорта из пригорода всего за 399 рублей', SPAM], 
#     ['Добро пожаловать в Мой Круг', NOT_SPAM],  
#     ['Я все еще жду документы', NOT_SPAM],  
#     ['Приглашаем на конференцию Data Science', NOT_SPAM],
#     ['Потерял твой телефон напомни', NOT_SPAM],
#     ['Порадуй своего питомца новым костюмом', SPAM]
# ]
train_data = pd.read_csv('spam_or_not_spam.csv')
# aug_data = pd.read_csv('spam_ham_dataset.csv', usecols=['label', 'text'])
# aug_data.columns=['label', 'email']
# aug_data = aug_data[['email', 'label']]
# labels = {
#     'spam': True,
#     'ham': False,
#          }
# aug_data['label'].map(labels)
# aug_data['email'] = aug_data['email'].apply(lambda text: text[text.find('\r\n')+2:])
# train_data = pd.concat([train_data, aug_data])

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [6]:
train_data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [7]:
train_data.dropna(inplace=True)

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [9]:
labels = {
    0: NOT_SPAM,
    1: SPAM,
}

In [10]:
train_data['label'] = train_data['label'].map(labels)

In [11]:
# Создадим функцию, приводящую текст к нижнему регистру и очищающую его от знаков пунктуации
def get_words(text):
    '''
        Функция get_words(text) возвращает список слов текста в нижнем регистре, игнорируя знаки пунктуации.
        Аргументы:
            text - строка с текстом
        Результат:
            список слов (python list)
        Пример испольования:
            get_words('Hello, world!')
        вернет:
            ['hello', 'world']
    '''
    clean_text = text.lower()
    clean_text = ''.join(c for c in clean_text if c not in string.punctuation)
    words = clean_text.split()
    return words

In [12]:
def calculate_word_frequencies(body, label):
    '''
        Функция calculate_word_frequencies(body, label) заполняет словари
        spam_words и good_words, содержащие данные о количестве вхождений
        слов в тексты, помеченные как спам и не спам соответственно.
        Также эта функция обновляет справочник количеств слов words_count.
        Аргументы:
            body - текст
            label - один из двух вариантов констант SPAM (равносильно True)
            и NOT_SPAM (равносильно False)
        Возврат:
            Функция ничего не возвращает. Только меняет глобальные переменные
            spam_words, good_words, words_count.
    '''
    
    # Получим списки слов из текста
    words = get_words(body)
    
    # Обновим дaнные в счетчиках
    if label == SPAM:
        # Обновим счетчик текстов, содержащих спам
        probs['spam_texts_count'] += 1
        
        # Обновим счетчик слов сообщений, содержащих спам
        words_count['spam'] += len(words)
        
        # Обновим счетчики для каждого слова
        for word in words:
            try:
                spam_words[word] += 1
            except:
                spam_words[word] = 1
    else:
        # Обновим счетчик текстов, не содержащих спам
        probs['good_texts_count'] += 1
        
        # Обновим счетчик слов сообщений, не содержащих спам
        words_count['good'] += len(words)
        
        # Обновим счетчики для каждого слова
        for word in words:
            try:
                good_words[word] += 1
            except:
                good_words[word] = 1    

In [13]:
def train(train_data):
    # Заполним справочники
    for data in train_data:
        calculate_word_frequencies(*data)
    
    # Вычислим вероятности
    texts_count = probs['spam_texts_count'] + probs['good_texts_count']
    probs['pA'] = np.longdouble(probs['spam_texts_count']) / np.longdouble(texts_count)
    probs['pNotA'] = np.longdouble(probs['good_texts_count']) / np.longdouble(texts_count)

In [14]:
train(zip(train_data['email'], train_data['label']))

In [15]:
probs

{'spam_texts_count': 499,
 'good_texts_count': 2500,
 'pA': 0.16638879626542180726,
 'pNotA': 0.83361120373457819275}

In [16]:
def calculate_P_Bi_A(word, label):
    '''
        Функция calculate_P_Bi_A(word, label) возвращает вероятность того,
        что слово word встретится в сообщении содержащем спам, при label == SPAM,
        или вероятность того, что слово встретится в сообщении не содержащем спам,
        при label != SPAM
    '''
    if label == SPAM:
        try:
            word_count = spam_words[word]
        except:
            word_count = 1
            spam_words[word] = 1
#             words_count['spam'] += 1
        return np.longdouble(word_count) / np.longdouble(words_count['spam'])
    else:
        try:
            word_count = good_words[word]
        except:
            word_count = 1
            good_words[word] = 1
#             words_count['good'] += 1
        return np.longdouble(word_count) / np.longdouble(words_count['good'])

In [17]:
def calculate_P_B_A(text, label):
    '''
        Функция calculate_P_B_A(text, label) возвращает вероятность спама для строки text
        при label == SPAM и вероятность не спама в противном случае
    '''
    # Сначала получим список слов
    words = get_words(text)
    
    # Вычисляем общую вероятность перемножением вероятностей по каждому слову
    # Что равнозначно сложению натуральных логарифмов вероятностей
    prob_ln = 0
    for word in words:
        prob_ln += np.log(calculate_P_Bi_A(word, label))
    
    return prob_ln

In [18]:
def classify(email):
    spam_prob_ln = calculate_P_B_A(email, SPAM) + np.log(np.longdouble(probs['pA']))
    not_spam_prob_ln = calculate_P_B_A(email, NOT_SPAM) + np.log(np.longdouble(probs['pNotA']))
    return spam_prob_ln > not_spam_prob_ln

In [19]:
train_data['spam'] = train_data['email'].apply(classify)

In [20]:
train_data.head()

,email,label,spam
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,False,False
1,martin a posted tassos papadopoulos the greek ...,False,False
2,man threatens explosion in moscow thursday aug...,False,False
3,klez the virus that won t die already the most...,False,False
4,in adding cream to spaghetti carbonara which ...,False,False


In [21]:
accuracy = sum(train_data['label'] == train_data['spam']) / len(train_data)
print(f'Точность модели: {accuracy:.2%}')

Точность модели: 99.17%


In [22]:
# Previous accuracy = 99.17%

In [23]:
text1 = 'Hi, My name is Warren E. Buffett an American business magnate, investor and philanthropist. am the most successful investor in the world. I believe strongly in‘giving while living’ I had one idea that never changed in my mind? that you should use your wealth to help people and i have decided to give {$1,500,000.00} One Million Five Hundred Thousand United Dollars, to randomly selected individuals worldwide. On receipt of this email, you should count yourself as the lucky individual. Your email address was chosen online while searching at random. Kindly get back to me at your earliest convenience before i travel to japan for my treatment , so I know your email address is valid. Thank you for accepting our offer, we are indeed grateful You Can Google my name for more information: God bless you. Best Regard Mr.Warren E. Buffett Billionaire investor !'
text2 = "Hi guys I want to build a website like REDACTED and I wanted to get your perspective of whether that site is good from the users' perspective before I go ahead and build something similar. I think that the design of the site is very modern and nice but I am not sure how people would react to a similar site? I look forward to your feedback. Many thanks!"
text3 = 'As a result of your application for the position of Data Engineer, I would like to invite you to attend an interview on May 30, at 9 a.m. at our office in Washington, DC. You will have an interview with the department manager, Moris Peterson. The interview will last about 45 minutes. If the date or time of the interview is inconvenient, please contact me by phone or email to arrange another appointment. We look forward to seeing you. '

In [24]:
classify(text1)

True

In [25]:
classify(text2)

False

In [26]:
classify(text3)

True

In [27]:
calculate_P_B_A(text2, NOT_SPAM)

-450.77445131479660262

In [28]:
calculate_P_B_A(text2, SPAM)

-467.5232691957518436